In [ ]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# Read the library CSV file
market_df = pd.read_csv('../data/raw/external/victoria_libraries.csv')
market_df = market_df.dropna(subset=['longitude', 'latitude'])

# Function to create GeoJSON Features
def create_geojson_feature(row, feature_type='library'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # Modify the 'address' column as needed
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# Convert library data to GeoJSON Features
market_features = market_df.apply(lambda row: create_geojson_feature(row, 'library'), axis=1).tolist()
market_feature_collection = FeatureCollection(market_features)

# Save library data as a GeoJSON file
houses_geojson_output_path = '../data/raw/external/API/1/libraries_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(market_feature_collection, f)

print("The GeoJSON file for libraries has been generated")


In [ ]:
import geojson
import openrouteservice
import pandas as pd
import time

# Initialize OpenRouteService API client
api_key = '5b3ce3597851110001cf6248dda07331ae64496fa25c4e54b59fba30' 
client = openrouteservice.Client(key=api_key)

# Read GeoJSON files
with open('../data/raw/external/API/1/property_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/1/libraries_geojson.geojson', 'r') as f:
    libraries_geojson = geojson.load(f)

# Extract latitude and longitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
library_coordinates = extract_coordinates(libraries_geojson)

# Extract house and library coordinates
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
library_coords = [(lon, lat) for lat, lon, _ in library_coordinates]

# Use OpenRouteService API to calculate distances in batches
def calculate_distance_matrix_in_batches(house_coords, library_coords, batch_size=20):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + library_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(library_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # Process the distances between each house and library, finding the shortest
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # Exclude None values and find the shortest distance
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_library_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_library = library_coordinates[closest_library_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_library': closest_library,
                        'min_driving_distance_km': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # Prevent rate-limiting
    
    return all_shortest_distances

# Process in batches
shortest_distances = calculate_distance_matrix_in_batches(house_coords, library_coords, batch_size=20)

# Save the shortest distance results to a CSV file
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/1/closest_library_distance_final_1.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"Results have been saved to {output_path}")
